In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import csv, sys, os
from tqdm import tqdm
import binascii, bitstring
from scipy.signal import find_peaks


def Parse_Float16(binfloat:str):
    """
    Turns string representation of 16 bits into a half-precision float.
    Assumes IEEE 754-2008 standard, 1 sign bit, 5 exponent bits, 10 mantissa bits.
    """
    S = { '0':1.0, '1':-1.0  }  # Map sign bit string to multiplicand

    sign = binfloat[0]
    exp = binfloat[1:6]
    mantissa = binfloat[6:]
   
    if exp=="00000":
        if mantissa=="0000000000": # Underflow
            value = 0.0
        else:
            value = S[sign] * 2**(-14) * (int(mantissa, 2) * 2**(-10))

    elif exp=="11111":         # Overflow case
        if sign=='0':
            value = np.NaN     # Currently set to Nan, this could be np.inf
        elif sign=='1':
            value = np.NaN     # Currently set to Nan, this could be -np.inf

    else:
        fraction = 1.0 + (int(mantissa, 2) * 2**(-10))
        value = S[sign] * fraction * 2**(int(exp, 2) - 15)    # 15 is half-precision bias

    return value

In [ ]:
x, y, z = [], [], []
with open('data/test1.log', 'r') as file:
    accel_data = []
    for line in tqdm(file):
        squished = line.replace(" ", "")
        addr, data = squished.split("|")
        

        segments = []
        for i in range(0, len(data)-1, 4):
            segments.append(data[i:i + 4])

        accel_data += segments

    for i in range(len(accel_data)):
        T = bitstring.BitArray(uint=int(accel_data[i], 16), length=16).bin
        if i%3 == 0:
            x.append( Parse_Float16(T) )
        elif i%3 == 1:
            y.append( Parse_Float16(T) )
        elif i%3 == 2:
            z.append( Parse_Float16(T) )

In [ ]:
temp1, temp2 = [], []
with open('data/temp_test1.txt', 'r') as file:
    temp_data = []
    for line in file:
        squished = line.replace(" ", "")
        addr, data = squished.split("|")

        segments = []
        for i in range(0, len(data)-1, 4):
            segments.append(data[i:i + 4])

        temp_data += segments

    for i in range(len(temp_data)):
        print(i)
        T = bitstring.BitArray(uint=int(temp_data[i], 16), length=16).bin
        if i%2 == 0:
            temp1.append( Parse_Float16(T) )
        elif i%2 == 1:
            temp2.append( Parse_Float16(T) )

In [ ]:
# Plot three axis
fig, ax = plt.subplots(5, 1, figsize=(9.5, 13.5) )

# make array for 50 hz x-axis
xt = np.arange(0, len(x)/50.0, 1/50.0)
yt = np.arange(0, len(y)/50.0, 1/50.0)
zt = np.arange(0, len(z)/50.0, 1/50.0)

for a_ in ax:
    a_.set_xlabel('Time (s)')


ax[0].plot(xt, x, label='x', color='#ed0000')
ax[1].plot(yt, y, label='y', color='#00a600')
ax[2].plot(zt, z, label='z', color='#0000ed')
ax[0].set_title('X Acceleration')
ax[1].set_title('Y Acceleration')
ax[2].set_title('Z Acceleration')
ax[0].set_ylabel('Acceleration (g)'); ax[1].set_ylabel('Acceleration (g)'); ax[2].set_ylabel('Acceleration (g)')

ax[3].plot(temp1, label='temp1', color="#4500a6")
ax[4].plot(temp2, label='temp2', color="#8d00d4")
ax[3].set_title('Temperature 1')
ax[4].set_title('Temperature 2')
ax[3].set_ylabel('Temperature (C)'); ax[4].set_ylabel('Temperature (C)')

fig.subplots_adjust(hspace=0.5)